## Download Vault from Hashicorp from https://developer.hashicorp.com/vault/downloads

## Start the dev server using the below command in PS Terminal

D:\\software\\vault\\vault.exe server -dev

## Add VAULT_ADDR to env variable from the terminal output

In [15]:
import os
os.environ["VAULT_ADDR"] = "http://127.0.0.1:8200"
print(os.getenv("VAULT_ADDR"))

http://127.0.0.1:8200


# Check status of vault

In [17]:
!"D:\\software\\vault\\vault.exe" status

Key             Value
---             -----
Seal Type       shamir
Initialized     true
Sealed          false
Total Shares    1
Threshold       1
Version         1.13.2
Build Date      2023-04-25T13:02:50Z
Storage Type    inmem
Cluster Name    vault-cluster-7babfc94
Cluster ID      ef242f65-e416-aecb-ab4c-3edb51dce6d5
HA Enabled      false


In [ ]:
!pip install hvac

In [77]:
# This PoC demonstrates role-based 
# access for key-value pair secrets as example 
import os 
import hvac 

# Authentication 
client = hvac.Client( 
    url='http://127.0.0.1:8200', 
    token=os.environ.get('VAULT_TOKEN'), 
) 
 
# Define the policy for user 1 
policy_name_1 = 'user-1-policy' 
policy_rules_1 = """ 
path "secret/*" { 
  capabilities = ["list"] 
} 
 
path "secret/foo/*" { 
  capabilities = ["list", "read"] 
} 
""" 
# Create the policy 
response = client.sys.create_or_update_policy(policy_name_1, policy_rules_1) 
 
# Define the policy for user 2 
policy_name_2 = 'user-2-policy' 
policy_rules_2 = """ 
path "secret/*" { 
  capabilities = ["list"] 
} 
 
path "secret/bar/*" { 
  capabilities = ["list", "read"] 
} 
""" 
# Create the policy 
response = client.sys.create_or_update_policy(policy_name_2, policy_rules_2) 
 
 
# Enable userpass engine for UI access 
# Check if the userpass auth method is already enabled 
auth_methods = client.sys.list_auth_methods()['data'] 
if 'userpass/' not in auth_methods: 
    # Enable the userpass auth method 
    client.sys.enable_auth_method('userpass') 
 
# Assign the policy to a test user1 
# Password same as username 
username = 'user1' 
password = username 
response = client.auth.userpass.create_or_update_user( 
    username=username, 
    password=password, 
    policies=[policy_name_1], 
) 
 
# Assign the policy to a test user1 
# Password same as username 
username = 'user2' 
password = username 
response = client.auth.userpass.create_or_update_user( 
    username=username, 
    password=password, 
    policies=[policy_name_2], 
) 
 
# Create secrets in specified paths 
create_response = client.secrets.kv.v2.create_or_update_secret( 
    path='foo/my', 
    secret=dict(password='secret1'), 
) 
 
create_response = client.secrets.kv.v2.create_or_update_secret( 
    path='bar/my', 
    secret=dict(password='secret2'), 
) 
 

In [85]:
# Test case #1 - user 1 - check list access
# login as user 1
user1_client = hvac.Client( 
    url='http://127.0.0.1:8200', )
user1_client.auth.userpass.login(
    username="user1",
    password="user1",
)
print(user1_client.is_authenticated()) 

list_response = user1_client.secrets.kv.v2.list_secrets(
    path='foo',
)
print(list_response)

# Test case #2 - user 1 - check read access
try:
    secrets_list = user1_client.secrets.kv.v2.read_secret(
        path = 'foo/my'
    )
    print(secrets_list)
    # Password is accessible because the policy has read capability
    # https://developer.hashicorp.com/vault/docs/concepts/policies
    print(secrets_list["data"]["data"])
except Exception as e:
    exception = str(e).replace("\n", "")
    print(f"Caught exception {exception}")

# Test case #3 - user 1 - check write access
try:
    create_response = user1_client.secrets.kv.v2.create_or_update_secret(
        path='foo/my',
        secret=dict(password='sample123123'),
    )
except Exception as e:
    exception = str(e).replace("\n", "")
    print(f'Caught exception {exception}')



True
{'request_id': '65544f14-fdd5-dafe-b452-cc9dcbf98726', 'lease_id': '', 'renewable': False, 'lease_duration': 0, 'data': {'keys': ['my', 'sample']}, 'wrap_info': None, 'warnings': None, 'auth': None}
Caught exception 1 error occurred:	* permission denied, on get http://127.0.0.1:8200/v1/secret/data/foo/my
Caught exception 1 error occurred:	* permission denied, on post http://127.0.0.1:8200/v1/secret/data/foo/my


In [86]:
# Test case #4 - user 2 - check list access
# login as user 2
user2_client = hvac.Client( 
    url='http://127.0.0.1:8200', )
user2_client.auth.userpass.login(
    username="user2",
    password="user2",
)
print(user2_client.is_authenticated()) 

list_response = user2_client.secrets.kv.v2.list_secrets(
    path='bar',
)
print(list_response)
# Test case #5 - user 2 - check read access
try:
    secrets_list = user2_client.secrets.kv.v2.read_secret(
        path = 'bar/my'
    )
    print(secrets_list)
    # Password is accessible because the policy has read capability
    # https://developer.hashicorp.com/vault/docs/concepts/policies
    print(secrets_list["data"]["data"])
except Exception as e:
    exception = str(e).replace("\n", "")
    print(f"Caught exception {exception}")

# Test case #6 - user 2 - check write access
try:
    create_response = user2_client.secrets.kv.v2.create_or_update_secret(
        path='bar/my',
        secret=dict(password='Hashi123'),
    )
except Exception as e:
    exception = str(e).replace("\n", "")
    print(f"Caught exception {exception}")


True
{'request_id': 'bfd375fc-9bbf-3612-8a61-6d3e53e53e61', 'lease_id': '', 'renewable': False, 'lease_duration': 0, 'data': {'keys': ['my']}, 'wrap_info': None, 'warnings': None, 'auth': None}
Caught exception 1 error occurred:	* permission denied, on get http://127.0.0.1:8200/v1/secret/data/bar/my
Caught exception 1 error occurred:	* permission denied, on post http://127.0.0.1:8200/v1/secret/data/bar/my
